<a href="https://colab.research.google.com/github/WeiZhenLim/2X/blob/main/AirTable_LinkedIn_Data_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>